# Linux Kernel Module Parameters

The Linux kernel provides a simple framework, enabling drivers to declare parameters that the user can specify on either boot or module load and then have these parameters exposed in your driver as global variables.These module parameters also show up in `sysfs` system.

## 1. Loading a module with parameters
We're going the write a demo kernel module, trying to pass parameters to the module during loading.
```sh
insmod ./hello-parameters.ko irq=20 name=mydev debug=1
```
We start with the following 'Hello, World!' source file:

In [1]:
%%writefile hello-parameters.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/moduleparam.h>

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Clymber Loong");
MODULE_DESCRIPTION("A Hello, World Module");

static int   irq   = 10;
static int   debug = 0;
static char *name  = "Hello World"; 

static int hello_init(void)
{
    printk(KERN_ALERT "Hello, World! name=%s, irq=%d, debug=%d\n", name, irq, debug);
    return 0;
}

static void hello_exit(void)
{
    printk(KERN_ALERT "Bye Bye, World!\n");
}

module_init(hello_init);
module_exit(hello_exit);

Overwriting hello-parameters.c


### 1.1 Defining parameters

Defining a module parameter is done via `the macro module_param()`:
```c
// @name: both the parameter exposed to the user and the variable holding
//        the parameter inside your module;
// @type: holds the parameter’s data type;
// @perm: specifies the permissions of the corresponding file in sysfs.
module_param(name, type, perm);
```
The `type` argument holds the parameter’s data type; it is one of byte, short, ushort, int, uint, long, ulong, charp, bool, or invbool.  
These types are, respectively, a byte, a short integer, an unsigned short integer, an integer, an unsigned integer, a long integer, an unsigned long integer, a pointer to a char, a Boolean, and a Boolean whose value is inverted from what the user specifies.The byte type is stored in a single char and the Boolean types are stored in variables of type int.

In [2]:
%%writefile -a hello-parameters.c
module_param(irq, int, 0660);
module_param(debug, int, 0660);
module_param(name, charp, 0660);

Appending to hello-parameters.c


### 1.2 Documenting the parameters (Optional)

We can document your parameters by using `MODULE_PARM_DESC()`:

In [3]:
%%writefile -a hello-parameters.c

MODULE_PARM_DESC(irq, "The interrupt line number.");
MODULE_PARM_DESC(debug, "Debug mode, 1 is on, 0 is off.");
MODULE_PARM_DESC(name, "Your name");

Appending to hello-parameters.c


### 1.2 Passing parameters during loading module

In [4]:
! make

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/module_param modules
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CC [M]  /home/clymber/Studio/kerdevel/module_param/hello-parameters.o
  LD [M]  /home/clymber/Studio/kerdevel/module_param/module_param.o
  MODPOST /home/clymber/Studio/kerdevel/module_param/Module.symvers
  CC [M]  /home/clymber/Studio/kerdevel/module_param/module_param.mod.o
  LD [M]  /home/clymber/Studio/kerdevel/module_param/module_param.ko
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'


In [5]:
%%script bash
sudo dmesg -C
sudo insmod ./*.ko name='Clymber' irq=100 debug=1
sudo dmesg -c

[ 1996.929443] Hello, World! name=Clymber, irq=100, debug=1


### 1.3 Verify the result

1. `/proc/modules` shows what kernel modules (drivers) are currently loaded into memory.
2. These module parameters also show up in `sysfs` system.

In [6]:
! grep 'hello' /proc/modules

In [7]:
! ls -l /sys/module/module_param/parameters/

total 0
-rw-rw---- 1 root root 4096 May 13 19:05 debug
-rw-rw---- 1 root root 4096 May 13 19:05 irq
-rw-rw---- 1 root root 4096 May 13 19:05 name


### 1.4 Cleaning up

In [9]:
%%script bash
sudo rmmod ./*.ko
sudo dmesg
make clean

[ 1997.303060] Bye Bye, World!
make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/module_param clean
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CLEAN   /home/clymber/Studio/kerdevel/module_param/Module.symvers
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'
